In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

# Load Data
train_df = pd.read_csv('training.csv')
test_df = pd.read_csv('test_X.csv')

X_train = train_df.drop(columns=['lettr'])
X_test = test_df.copy()

# Normalize Data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build Deep Autoencoder
input_dim = X_train_scaled.shape[1]

input_layer = Input(shape=(input_dim,))
encoded = Dense(64, activation='relu')(input_layer)
encoded = BatchNormalization()(encoded)
encoded = Dense(32, activation='relu')(encoded)
encoded = Dropout(0.2)(encoded)
encoded = Dense(16, activation='relu')(encoded)

decoded = Dense(32, activation='relu')(encoded)
decoded = BatchNormalization()(decoded)
decoded = Dense(64, activation='relu')(decoded)
decoded = Dropout(0.2)(decoded)
output_layer = Dense(input_dim, activation='linear')(decoded)

autoencoder = Model(inputs=input_layer, outputs=output_layer)
autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

# Train Model with Early Stopping
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

autoencoder.fit(
    X_train_scaled, X_train_scaled,
    epochs=150,
    batch_size=64,
    shuffle=True,
    validation_split=0.1,
    callbacks=[early_stop],
    verbose=1
)

# Compute Reconstruction Errors
reconstructed = autoencoder.predict(X_test_scaled)
mse = np.mean(np.square(X_test_scaled - reconstructed), axis=1)

# Create Submission
submission = pd.DataFrame({
    'id': np.arange(len(mse)),
    'outliers': mse
})

submission.to_csv('submission.csv', index=False)
print("saved as 'submission.csv'")


Epoch 1/150
60/60 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 1.2520 - val_loss: 1.2353
Epoch 2/150
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.6449 - val_loss: 0.9108
Epoch 3/150
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.4850 - val_loss: 0.6498
Epoch 4/150
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.4271 - val_loss: 0.5109
Epoch 5/150
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.3956 - val_loss: 0.4014
Epoch 6/150
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.3709 - val_loss: 0.3560
Epoch 7/150
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.3436 - val_loss: 0.3230
Epoch 8/150
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.3261 - val_loss: 0.3039
Epoch 9/150
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.3164 - val_loss: 0.2757
Epoch 10/150
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.3041 - val_loss: 0.2592
Epoch 11/150
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2973 - val_loss: 0.2485
Epoch 12/150
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2